In [24]:
import pandas as pd
import json
import re

In [19]:
monosemy_raw_df = pd.read_csv("output_0.7_mono_seeded.csv")

In [21]:
monosemy_raw_df= monosemy_raw_df.drop(columns = ['Unnamed: 0', 'llama2 output 1'])
monosemy_raw_df[:3]

,verb,llama_mono_output
0,abuse,polysemous
1,allow,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...
2,approve,polysemous


In [73]:
monosemy_raw_df = monosemy_raw_df[monosemy_raw_df['llama_mono_output']!="polysemous"]
monosemy_raw_df[:10]

,verb,llama_mono_output
1,allow,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...
5,box,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...
9,button,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...
12,chip,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...
13,chop,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...
14,chuck,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...
15,clean,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...
16,clear,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...
17,coat,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...
22,earn,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...


In [158]:
def get_substring_after(string, substring):
  sense_descr = string
  if substring in string:
      start_index = sense_descr.find(substring)
      if start_index != -1:
          start_index += len(substring)
          end_index = sense_descr.find(".", start_index)
          if end_index != -1:
              sense_descr = sense_descr[start_index:end_index].strip()
  return sense_descr

def remove_sys_strings(input_string):
    substrings_to_remove = ["</sys>", "<sys>", "<</sys>"]

    for substring in substrings_to_remove:
        input_string = input_string.replace(substring, "")

    return input_string.strip()

In [163]:
def extract_sense(verb, output):
  # extract text after [/INST]
  match = re.search(r'\[/INST\](.*?)</s>', output, re.DOTALL)
  if match:
      sense_descr = match.group(1).strip().lower()
  else:
    sense_descr = output.lower()

  sense_descr = remove_sys_strings(sense_descr)

  # get text before "example"
  if "example" in sense_descr or "examples" in sense_descr:
    match = re.search(r'(.*?)example|examples', sense_descr, re.DOTALL)
    sense_descr = match.group(0).strip()

  sense_descr = get_substring_after(sense_descr, "it typically means ")
  sense_descr = get_substring_after(sense_descr, "it means ")
  sense_descr = get_substring_after(sense_descr, "can mean ")
  sense_descr = get_substring_after(sense_descr, "to describe ")
  sense_descr = get_substring_after(sense_descr, "one possible sense when used in a transitive clause: 1.")
  sense_descr = get_substring_after(sense_descr, "sense: ")
  sense_descr = get_substring_after(sense_descr, "one possible sense is: 1. ")
  sense_descr = get_substring_after(sense_descr, "in the following sense: ")
  sense_descr = get_substring_after(sense_descr, "1. ")
  sense_descr = get_substring_after(sense_descr, "the most common sense is")
  sense_descr = get_substring_after(sense_descr, "in the following sense:")
  sense_descr = get_substring_after(sense_descr, "means")

  verb_string = "the most common sense of \"{}\" is".format(verb)
  sense_descr = get_substring_after(sense_descr, verb_string)

  verb_string = "sense of {} is ".format(verb)
  sense_descr = get_substring_after(sense_descr, verb_string)

  result = re.sub(r'[^a-zA-Z\s]', '', sense_descr)

  return result.strip()

In [142]:
test_string = "<<sys>> the verb \"chuck\" can have one possible sense when used in a transitive clause: 1. to throw or hurl something away or out of one's hand or from a place. example"
print(extract_sense("chuck", test_string))

to throw or hurl something away or out of one's hand or from a place


In [164]:
monosemy_df = monosemy_raw_df.copy().reset_index(drop=True)
monosemy_df['sense_descr'] = monosemy_df.apply(lambda row: extract_sense(row['verb'], str(row['llama_mono_output'])), axis=1)
monosemy_df

,verb,llama_mono_output,sense_descr
0,allow,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...,to give permission or approval for something t...
1,box,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...,to pack or contain something in a box
2,button,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...,the act of pressing a button
3,chip,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...,the act of breaking or cracking something into...
4,chop,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...,to cut or slice something using a sharp instru...
5,chuck,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...,to throw or hurl something away or out of ones...
6,clean,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...,to remove dirt stains or impurities from a sur...
7,clear,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...,to remove something that is blocking or obscur...
8,coat,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...,to cover or layer something with a substance s...
9,earn,<s> [INST] <<SYS>>\n \n <</SYS>>\n Pl...,to gain or receive something in return for one...


In [165]:
monosemy_df.to_csv("mono_sense_descr.csv")